In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils import globalsettings as gs

In [2]:
# Cargando el archivo .npz
data_npz = np.load(gs.the_files.DATA_LIGHTWEIGHT)

# Obtener los nombres de los arrays
array_names = data_npz.files
array_names


['media_data', 'extra_features', 'costs']

In [3]:
# Cargando los arrays en variables con los mismos nombres
media_data = data_npz['media_data']
extra_features = data_npz['extra_features']
costs = data_npz['costs']

# Verificando las dimensiones de los arrays cargados
media_data.shape, extra_features.shape, costs.shape


((160, 3, 5), (160, 2, 5), (3,))

In [4]:
media_data_2d = media_data.reshape(160, -1)

# Supongamos que estos son los valores para 'media_channel' y 'geo'
media_channels = ['channel1', 'channel2', 'channel3']
geos = ['geo1', 'geo2', 'geo3', 'geo4', 'geo5']

# Crea las columnas basadas en las combinaciones de 'media_channel' y 'geo'
columns = [f'{channel}_{geo}' for channel in media_channels for geo in geos]

# Crea el DataFrame
df_wide = pd.DataFrame(media_data_2d, columns=columns, index = pd.date_range(start='2022-01-03', periods=160, freq='W-MON'))

df_wide.head()

,channel1_geo1,channel1_geo2,channel1_geo3,channel1_geo4,channel1_geo5,channel2_geo1,channel2_geo2,channel2_geo3,channel2_geo4,channel2_geo5,channel3_geo1,channel3_geo2,channel3_geo3,channel3_geo4,channel3_geo5
2022-01-03,20.309933,20.309933,20.309933,20.309933,20.309933,21.495794,21.495794,21.495794,21.495794,21.495794,23.264574,23.264574,23.264574,23.264574,23.264574
2022-01-10,20.574215,20.574215,20.574215,20.574215,20.574215,19.579594,19.579594,19.579594,19.579594,19.579594,21.324257,21.324257,21.324257,21.324257,21.324257
2022-01-17,18.423088,18.423088,18.423088,18.423088,18.423088,22.889912,22.889912,22.889912,22.889912,22.889912,20.188976,20.188976,20.188976,20.188976,20.188976
2022-01-24,19.161734,19.161734,19.161734,19.161734,19.161734,20.355442,20.355442,20.355442,20.355442,20.355442,20.265499,20.265499,20.265499,20.265499,20.265499
2022-01-31,20.657064,20.657064,20.657064,20.657064,20.657064,21.434008,21.434008,21.434008,21.434008,21.434008,21.574446,21.574446,21.574446,21.574446,21.574446


In [5]:
# Convirtiendo el DataFrame de formato ancho a formato largo
df_long = df_wide.reset_index().melt(id_vars='index', value_vars=columns, var_name='media_geo', value_name='value')

# Separando las columnas 'media_channel' y 'geo' a partir de la columna 'media_geo'
df_long[['media_channel', 'geo']] = df_long['media_geo'].str.split('_', expand=True)

# Renombrando la columna 'index' a 'date'
df_long.rename(columns={'index': 'date'}, inplace=True)

# Eliminando la columna 'media_geo' ya que ya hemos extraído la información en 'media_channel' y 'geo'
df_long.drop(columns=['media_geo'], inplace=True)

# Reordenando las columnas
df_long = df_long[['date', 'media_channel', 'geo', 'value']]

# Mostrando las primeras filas del DataFrame en formato largo
df_long.head()


,date,media_channel,geo,value
0,2022-01-03,channel1,geo1,20.309933
1,2022-01-10,channel1,geo1,20.574215
2,2022-01-17,channel1,geo1,18.423088
3,2022-01-24,channel1,geo1,19.161734
4,2022-01-31,channel1,geo1,20.657064


In [6]:
df_long.to_csv(gs.the_files.MEDIA_LONG, index = False)

In [7]:
# Reshape del array 3D a 2D (160, 3, 5) -> (160, 2 * 5)
extra_features_2d = extra_features.reshape(160, -1)

# Ajustando las características de acuerdo con la forma real del array
features = ['feature1', 'feature2']

# Creando las columnas basadas en las combinaciones de 'feature' y 'geo'
columns_extra = [f'{feature}_{geo}' for feature in features for geo in geos]

# Creando el DataFrame con formato ancho
df_extra_wide = pd.DataFrame(extra_features_2d, columns=columns_extra,
                             index=pd.date_range(start='2022-01-03', periods=160, freq='W-MON'))

# Convirtiendo el DataFrame de formato ancho a formato largo
df_extra_long = df_extra_wide.reset_index().melt(id_vars='index', value_vars=columns_extra, var_name='feature_geo', value_name='value')

# Separando las columnas 'feature' y 'geo' a partir de la columna 'feature_geo'
df_extra_long[['feature', 'geo']] = df_extra_long['feature_geo'].str.split('_', expand=True)

# Renombrando la columna 'index' a 'date'
df_extra_long.rename(columns={'index': 'date'}, inplace=True)

# Eliminando la columna 'feature_geo' ya que ya hemos extraído la información en 'feature' y 'geo'
df_extra_long.drop(columns=['feature_geo'], inplace=True)

# Reordenando las columnas
df_extra_long = df_extra_long[['date', 'feature', 'geo', 'value']]

# Mostrando las primeras filas del DataFrame en formato largo
df_extra_long.head()


,date,feature,geo,value
0,2022-01-03,feature1,geo1,4.242858
1,2022-01-10,feature1,geo1,4.760330
2,2022-01-17,feature1,geo1,5.610206
3,2022-01-24,feature1,geo1,6.484605
4,2022-01-31,feature1,geo1,4.094253


In [8]:
df_extra_long.to_csv(gs.the_files.EXTRA_LONG, index = False)

# Selección

1. Indexación Básica

Puedes acceder a un elemento específico en un array utilizando índices. Los índices comienzan en 0.

In [9]:
# Ejemplo de un Array 1D:

arr_1d = np.array([10, 20, 30, 40, 50])
element = arr_1d[2]  # Selecciona el tercer elemento

In [10]:
# Ejemplo de un Array 2D:

arr_2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
element = arr_2d[1, 2]  # Selecciona el tercer elemento de la segunda fila



2. Slicing

Puedes seleccionar una sección (rebanada) del array utilizando el operador de slicing :.

In [11]:
sliced_row = arr_2d[1, :]  # Selecciona toda la segunda fila
sliced_column = arr_2d[:, 1]  # Selecciona toda la segunda columna


3. Indexación Booleana

Puedes utilizar una condición para seleccionar elementos que cumplan ciertos criterios.

In [12]:
arr = np.array([1, 2, 3, 4, 5])
selected = arr[arr > 3]  # Selecciona elementos mayores que 3


4. Indexación de Matrices (Fancy Indexing)

Puedes utilizar listas o arrays de índices para seleccionar elementos específicos.

In [13]:
arr = np.array([10, 20, 30, 40, 50])
indices = [1, 3]
selected = arr[indices]  # Selecciona los elementos en las posiciones 1 y 3

# Operaciones

Operaciones aritméticas

Puedes realizar operaciones aritméticas elementales entre arrays del mismo tamaño. Esto incluye suma (+), resta (-), multiplicación (*), división (/), y potenciación (**).

In [14]:
import numpy as np

# Creando dos arrays numpy
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

# Suma
c = a + b
# Resultado: array([5, 7, 9])


Operaciones matemáticas

NumPy ofrece funciones matemáticas como sin, cos, exp, etc. Estas funciones se aplican elemento por elemento en el array.

In [15]:
# Creando un array numpy
a = np.array([1, 2, 3])

# Función exponencial
b = np.exp(a)
# Resultado: array([ 2.71828183,  7.3890561 , 20.08553692])


Operaciones de álgebra lineal

NumPy ofrece varias funciones en el submódulo numpy.linalg para realizar operaciones de álgebra lineal, como cálculo de determinantes, resolución de sistemas lineales, cálculo de autovalores y autovectores, etc.

In [16]:
# Creando dos arrays 2D
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7, 8]])

# Producto de matrices
c = np.dot(a, b)
# Resultado: array([[19, 22], [43, 50]])


Operaciones estadísticas

NumPy también ofrece funciones para calcular estadísticas de los datos en arrays, como min, max, sum, mean, median, std, etc

In [17]:
# Creando un array numpy
a = np.array([1, 2, 3, 4, 5])

# Media de los elementos del array
m = np.mean(a)
# Resultado: 3.0


Operaciones booleanas

Puedes aplicar operaciones booleanas elemento por elemento en arrays. Esto es útil para la indexación booleana y la selección condicional de elementos.

In [18]:
# Creando un array numpy
a = np.array([1, 2, 3, 4, 5])

# Creando una máscara booleana para valores mayores que 3
mask = a > 3
# Resultado: array([False, False, False,  True,  True])


# Broadcasting

Cuando los arrays en una operación no tienen las mismas dimensiones, NumPy intenta aplicar "broadcasting" para hacer que las dimensiones sean compatibles. El broadcasting es un potente mecanismo que permite que NumPy trabaje con arrays de diferentes formas cuando se realizan operaciones aritméticas.

Aquí están las reglas del broadcasting:

+ Regla 1: Si los dos arrays difieren en su número de dimensiones, la forma del que tiene menos dimensiones se rellena con unos en su lado izquierdo.
+ Regla 2: Si la forma de los dos arrays no coincide en alguna dimensión, el array con forma igual a 1 en esa dimensión se estira para coincidir con la otra forma.
+ Regla 3: Si en cualquier dimensión los tamaños difieren y ninguno es igual a 1, se genera un error.

In [19]:
import numpy as np

a = np.array([1, 2, 3]) # Forma (3,)
b = np.array([[10], [20], [30]]) # Forma (3, 1)

# a se estira a (3, 3) y b se estira a (3, 3)
c = a + b
# Resultado:
# array([[11, 12, 13],
#        [21, 22, 23],
#        [31, 32, 33]])


# Equivalencias con R

apply, sweep, aperm

apply 

La función apply en R es utilizada para aplicar una función a los márgenes de una matriz o array. En NumPy, puedes lograr un comportamiento similar utilizando la función np.apply_along_axis.

La función np.apply_along_axis aplica una función a lo largo del eje especificado de un array. La función toma tres argumentos principales: la función que quieres aplicar, el eje a lo largo del cual quieres aplicar la función, y el array en sí.

Aquí tienes un ejemplo que muestra cómo puedes usar np.apply_along_axis para replicar el comportamiento de apply en R:

In [20]:
import numpy as np

# Definiendo una función que quieres aplicar
def my_function(x):
    return x.mean()

# Creando un array 2D
array = np.array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])

# Aplicando la función a lo largo del eje 0 (columnas)
result_along_axis0 = np.apply_along_axis(my_function, axis=0, arr=array)
# Resultado: array([4., 5., 6.])

# Aplicando la función a lo largo del eje 1 (filas)
result_along_axis1 = np.apply_along_axis(my_function, axis=1, arr=array)
# Resultado: array([2., 5., 8.])


# Random y distribuciones

En NumPy, puedes establecer la semilla para la generación de números aleatorios utilizando numpy.random.seed(). Esta función asegura que los resultados aleatorios sean reproducibles. Es equivalente al set.seed() de R.

In [ ]:
np.random.seed(42)

Después de establecer la semilla, cualquier función que utilice la generación de números aleatorios de NumPy producirá los mismos resultados cada vez que se ejecute, siempre que se inicie con la misma semilla.

In [33]:
np.random.seed(42)
print(np.random.rand(5))  # Genera 5 números aleatorios entre 0 y 1

[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864]


1. Generación Básica de Números Aleatorios

a. Números Aleatorios Uniformes

Para generar números aleatorios entre 0 y 1 (distribución uniforme):

In [35]:
np.random.seed(42)
np.random.rand(5)  # Genera 5 números aleatorios entre 0 y 1

array([0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864])

b. Números Aleatorios Enteros

Equivalente a sample en R, puedes usar `np.random.choice` o `np.random.randint`:

In [36]:
np.random.choice(10, 5)  # Elige 5 números aleatorios entre 0 y 9
np.random.randint(0, 10, 5)  # Genera 5 números aleatorios entre 0 (inclusive) y 10 (exclusivo)


array([7, 7, 2, 5, 4])

2. Generación de Números Aleatorios de Distribuciones Estadísticas

NumPy proporciona funciones para generar números aleatorios de muchas distribuciones estadísticas comunes:

a. Distribución Normal (Gaussiana)

In [37]:
np.random.randn(5)  # 5 números aleatorios de una distribución normal estándar (media=0, desviación estándar=1)
np.random.normal(loc=0, scale=1, size=5)  # Especificando media (loc) y desviación estándar (scale)


array([ 0.95036968,  0.81644508, -1.523876  , -0.42804606, -0.74240684])

b. Distribución Binomial

In [38]:
n, p = 10, 0.5
np.random.binomial(n, p, 5)  # 5 números aleatorios de una distribución binomial con n intentos y probabilidad p

array([4, 5, 6, 4, 5])

c. Distribución Poisson

In [39]:
lambda_p = 3
np.random.poisson(lambda_p, 5)  # 5 números aleatorios de una distribución Poisson con lambda lambda_p


array([1, 2, 4, 2, 1])

d. Distribución Exponencial

In [40]:
scale = 1.0
np.random.exponential(scale, 5)  # 5 números aleatorios de una distribución exponencial


array([2.40042289, 0.29945778, 1.0862558 , 0.37354658, 0.7341109 ])

... y muchas otras distribuciones (gamma, beta, chi cuadrado, etc.)

3. Permutaciones y Mezcla de Arrays

a. Mezcla de Arrays (Shuffle)

In [41]:
arr = np.arange(10)
np.random.shuffle(arr)  # Mezcla el array en su lugar


b. Permutación de Arrays

In [42]:
arr = np.arange(10)
permuted_arr = np.random.permutation(arr)  # Devuelve un array permutado


# Ejemplo

In [21]:
# Cargando el primer archivo: "lightweight_data.npz"
import numpy as np

# Cargando el archivo
lightweight_data = np.load(gs.the_files.DATA_LIGHTWEIGHT)

# Obteniendo los nombres de los arrays dentro del archivo
lightweight_data_files = lightweight_data.files
lightweight_data_files


['media_data', 'extra_features', 'costs']

In [22]:
# Cargando el segundo archivo: "lightweight_traze.npz"
lightweight_traze = np.load(gs.the_files.TRAZE_LIGHTWEIGHT)

# Obteniendo los nombres de los arrays dentro del archivo
lightweight_traze_files = lightweight_traze.files
lightweight_traze_files


['channel_coef_media',
 'coef_extra_features',
 'coef_media',
 'coef_seasonality',
 'coef_trend',
 'expo_trend',
 'gamma_seasonality',
 'half_max_effective_concentration',
 'intercept',
 'lag_weight',
 'media_transformed',
 'mu',
 'sigma',
 'slope']

In [23]:
# Extrayendo los arrays
media_data = lightweight_data['media_data']
print(media_data.shape)

coef_media = lightweight_traze['coef_media']
print(coef_media.shape)

(160, 3, 5)
(2000, 3, 5)


In [24]:
# Inicializando una lista para almacenar los resultados parciales
result_list = []

# Iterando a través de cada componente 3x5 en coef_media y multiplicándolo elemento a elemento con media_data
for i in range(coef_media.shape[0]):
    partial_result = media_data * coef_media[i]
    result_list.append(partial_result)

# Apilando los resultados para obtener el array final
final_result = np.stack(result_list)

# Verificando la forma del resultado final
final_result_shape = final_result.shape
final_result_shape

# Si las dimensiones son i, j, k l:
# Tenemos 2000 "rebanadas", cada una es una muestra:
# - j = Fecha, k = medio, l = geo


(2000, 160, 3, 5)

In [25]:
# Sumando a lo largo de la tercera dimensión (axis=2) - medio
summed_result = np.sum(final_result, axis=2)

# Verificando la forma del resultado sumado
summed_result_shape = summed_result.shape
summed_result_shape

# Obtenemos las contribuciones de todos los medios
# Quedan 2000 muestras j = fecha, l = geo


(2000, 160, 5)

In [26]:
median_results = np.median(summed_result, axis = 0)
median_results.shape

# Obtenemos las contribuciones medianas
# j = fecha, i = geo

(160, 5)

In [27]:
coef_extra_features = lightweight_traze['coef_extra_features']
print(coef_extra_features.shape)

(2000, 2, 5)


In [28]:
coef_media_extra_features = np.concatenate([coef_media, coef_extra_features], axis = 1)
coef_media_extra_features.shape

(2000, 5, 5)

In [29]:
extra_features_data = lightweight_data['extra_features']
print(extra_features_data.shape)

(160, 2, 5)


In [30]:
media_extra_features_data = np.concatenate([media_data, extra_features_data], axis = 1)
print(media_extra_features_data.shape)

(160, 5, 5)
